## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [9]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session
from sqlalchemy import select

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

albums
['AlbumId', 'Title', 'ArtistId']
[('AlbumId', Column('AlbumId', INTEGER(), table=<albums>, primary_key=True, nullable=False)), ('Title', Column('Title', NVARCHAR(length=160), table=<albums>, nullable=False)), ('ArtistId', Column('ArtistId', INTEGER(), ForeignKey('artists.ArtistId'), table=<albums>, nullable=False))]
-------------------------
artists
['ArtistId', 'Name']
[('ArtistId', Column('ArtistId', INTEGER(), table=<artists>, primary_key=True, nullable=False)), ('Name', Column('Name', NVARCHAR(length=120), table=<artists>))]
-------------------------
customers
['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email', 'SupportRepId']
[('CustomerId', Column('CustomerId', INTEGER(), table=<customers>, primary_key=True, nullable=False)), ('FirstName', Column('FirstName', NVARCHAR(length=40), table=<customers>, nullable=False)), ('LastName', Column('LastName', NVARCHAR(length=20), table=<customers>, nullable=F

In [10]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

1 For Those About To Rock We Salute You


#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [3]:
### Imprima os três primeiros registros da tabela tracks
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)

# Selecionar os 3 primeiros registros da tabela tracks
query = select(Base.classes.tracks).limit(3)
result = session.execute(query).scalars().all()

# Imprimir os resultados
print("Três primeiros registros da tabela tracks:")
for track in result:
    print(f"TrackId: {track.TrackId}, Name: {track.Name}, AlbumId: {track.AlbumId}")

session.close()

Três primeiros registros da tabela tracks:
TrackId: 1, Name: For Those About To Rock (We Salute You), AlbumId: 1
TrackId: 2, Name: Balls to the Wall, AlbumId: 2
TrackId: 3, Name: Fast As a Shark, AlbumId: 3


In [4]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)

# Selecionar nome da faixa e título do álbum das primeiras 20 faixas
query = (
    select(Base.classes.tracks.Name, Base.classes.albums.Title)
    .join(Base.classes.albums, Base.classes.tracks.AlbumId == Base.classes.albums.AlbumId)
    .limit(20)
)
result = session.execute(query).all()

# Imprimir os resultados
print("Nome da faixa e título do álbum das primeiras 20 faixas:")
for row in result:
    print(f"Track: {row.Name}, Album: {row.Title}")

session.close()

Nome da faixa e título do álbum das primeiras 20 faixas:
Track: For Those About To Rock (We Salute You), Album: For Those About To Rock We Salute You
Track: Put The Finger On You, Album: For Those About To Rock We Salute You
Track: Let's Get It Up, Album: For Those About To Rock We Salute You
Track: Inject The Venom, Album: For Those About To Rock We Salute You
Track: Snowballed, Album: For Those About To Rock We Salute You
Track: Evil Walks, Album: For Those About To Rock We Salute You
Track: C.O.D., Album: For Those About To Rock We Salute You
Track: Breaking The Rules, Album: For Those About To Rock We Salute You
Track: Night Of The Long Knives, Album: For Those About To Rock We Salute You
Track: Spellbound, Album: For Those About To Rock We Salute You
Track: Balls to the Wall, Album: Balls to the Wall
Track: Fast As a Shark, Album: Restless and Wild
Track: Restless and Wild, Album: Restless and Wild
Track: Princess of the Dawn, Album: Restless and Wild
Track: Go Down, Album: Let Th

In [6]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)

# Selecionar as 10 primeiras vendas da tabela invoice_items
query = select(Base.classes.invoice_items).limit(10)
result = session.execute(query).scalars().all()

# Imprimir os resultados
print("10 primeiras vendas da tabela invoice_items:")
for item in result:
    print(f"InvoiceId: {item.InvoiceId}, TrackId: {item.TrackId}, UnitPrice: {item.UnitPrice}, Quantity: {item.Quantity}")

session.close()

### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
from sqlalchemy.orm import Session
from sqlalchemy import select

session = Session(engine)

# Selecionar nome da faixa e quantidade das 10 primeiras vendas
query = (
    select(Base.classes.tracks.Name, Base.classes.invoice_items.Quantity)
    .join(Base.classes.tracks, Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
    .limit(10)
)
result = session.execute(query).all()

# Imprimir os resultados
print("Nomes das faixas e quantidades das 10 primeiras vendas:")
for row in result:
    print(f"Track: {row.Name}, Quantity: {row.Quantity}")

session.close()


10 primeiras vendas da tabela invoice_items:
InvoiceId: 1, TrackId: 2, UnitPrice: 0.99, Quantity: 1
InvoiceId: 1, TrackId: 4, UnitPrice: 0.99, Quantity: 1
InvoiceId: 2, TrackId: 6, UnitPrice: 0.99, Quantity: 1
InvoiceId: 2, TrackId: 8, UnitPrice: 0.99, Quantity: 1
InvoiceId: 2, TrackId: 10, UnitPrice: 0.99, Quantity: 1
InvoiceId: 2, TrackId: 12, UnitPrice: 0.99, Quantity: 1
InvoiceId: 3, TrackId: 16, UnitPrice: 0.99, Quantity: 1
InvoiceId: 3, TrackId: 20, UnitPrice: 0.99, Quantity: 1
InvoiceId: 3, TrackId: 24, UnitPrice: 0.99, Quantity: 1
InvoiceId: 3, TrackId: 28, UnitPrice: 0.99, Quantity: 1
Nomes das faixas e quantidades das 10 primeiras vendas:
Track: Balls to the Wall, Quantity: 1
Track: Restless and Wild, Quantity: 1
Track: Put The Finger On You, Quantity: 1
Track: Inject The Venom, Quantity: 1
Track: Evil Walks, Quantity: 1
Track: Breaking The Rules, Quantity: 1
Track: Dog Eat Dog, Quantity: 1
Track: Overdose, Quantity: 1
Track: Love In An Elevator, Quantity: 1
Track: Janie's Go

In [7]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
from sqlalchemy.orm import Session
from sqlalchemy import select, func

session = Session(engine)

# Selecionar as 10 faixas mais vendidas
query = (
    select(Base.classes.tracks.Name, func.sum(Base.classes.invoice_items.Quantity).label("total_sold"))
    .join(Base.classes.invoice_items, Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
    .group_by(Base.classes.tracks.TrackId, Base.classes.tracks.Name)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
)
result = session.execute(query).all()

# Imprimir os resultados
print("10 faixas mais vendidas e quantas vezes foram vendidas:")
for row in result:
    print(f"Track: {row.Name}, Total Sold: {row.total_sold}")

session.close()

10 faixas mais vendidas e quantas vezes foram vendidas:
Track: Balls to the Wall, Total Sold: 2
Track: Inject The Venom, Total Sold: 2
Track: Snowballed, Total Sold: 2
Track: Overdose, Total Sold: 2
Track: Deuces Are Wild, Total Sold: 2
Track: Not The Doctor, Total Sold: 2
Track: Por Causa De Você, Total Sold: 2
Track: Welcome Home (Sanitarium), Total Sold: 2
Track: Snowblind, Total Sold: 2
Track: Cornucopia, Total Sold: 2


In [8]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
from sqlalchemy.orm import Session
from sqlalchemy import select, func

session = Session(engine)

# Selecionar os 10 artistas que mais venderam
query = (
    select(Base.classes.artists.Name, func.sum(Base.classes.invoice_items.Quantity).label("total_sold"))
    .join(Base.classes.albums, Base.classes.artists.ArtistId == Base.classes.albums.ArtistId)
    .join(Base.classes.tracks, Base.classes.albums.AlbumId == Base.classes.tracks.AlbumId)
    .join(Base.classes.invoice_items, Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
    .group_by(Base.classes.artists.ArtistId, Base.classes.artists.Name)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
)
result = session.execute(query).all()

# Imprimir os resultados
print("10 artistas que mais venderam:")
for row in result:
    print(f"Artist: {row.Name}, Total Sold: {row.total_sold}")

session.close()

10 artistas que mais venderam:
Artist: Iron Maiden, Total Sold: 140
Artist: U2, Total Sold: 107
Artist: Metallica, Total Sold: 91
Artist: Led Zeppelin, Total Sold: 87
Artist: Os Paralamas Do Sucesso, Total Sold: 45
Artist: Deep Purple, Total Sold: 44
Artist: Faith No More, Total Sold: 42
Artist: Lost, Total Sold: 41
Artist: Eric Clapton, Total Sold: 40
Artist: R.E.M., Total Sold: 39
